In [102]:
import pandas as pd
import numpy as np
import sys
sys.path.append('../src/')
import dataset_prep

Import de la base de données

In [103]:
selected_columns = [
"nb_packets_sent",
"distance",
"distRealSR1",
"pos_y_rec_f",
"pos_y_rec",
"pos_x_rec_f",
"pos_x_rec",
"pos_x_send",
"pos_y_send",
"spd_x_send",
"spd_y_send",
"sendTime",
"receiver",
"sender",
"label"
]

In [104]:
df = dataset_prep.import_dataset('../../data/0709/data0709.csv', selected_columns)

In [105]:
dataset_prep.clean_dataset(df)

Nombre de lignes avant nettoyage :  1048575
Nombre de lignes après nettoyage :  1038446


Analyse et vérification des données

In [106]:
senders = np.unique(df["sender"].values)
senders

array([   9,   15,   21, ..., 6687, 6693, 6699])

In [107]:
print(len(senders))

1116


In [108]:
df.loc[df['sender'] == 2613].sort_values("sendTime")

,sendTime,sender,receiver,pos_x_send,pos_y_send,spd_x_send,spd_y_send,label,pos_x_rec,pos_y_rec,pos_x_rec_f,pos_y_rec_f,nb_packets_sent,distRealSR1,distance
350483,26171.540601,2613,2529,111.335251,891.396480,1.431214,0.122446,13,146.516620,900.286134,146.679510,900.281326,220,35.181369,36.287114
350399,26171.540601,2613,2439,111.335251,891.396480,1.431214,0.122446,13,151.941293,900.330041,151.871047,900.362848,220,40.606042,41.577147
350441,26171.540601,2613,2481,111.335251,891.396480,1.431214,0.122446,13,154.252143,900.763757,154.404746,900.756218,220,42.916892,43.927275
350484,26171.790600,2613,2529,111.335251,891.396480,1.431214,0.122446,13,146.516620,900.286134,146.679510,900.281326,220,35.181369,36.287114
350400,26171.790600,2613,2439,111.335251,891.396480,1.431214,0.122446,13,151.941293,900.330041,151.871047,900.362848,220,40.606042,41.577147
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
350524,26181.790610,2613,2529,134.099307,955.698874,-0.243714,3.314662,13,147.184252,900.253071,147.317005,900.234151,220,13.084945,56.968876
350440,26181.790610,2613,2439,134.099307,955.698874,-0.243714,3.314662,13,151.429332,900.407391,151.344435,900.400718,220,17.330025,57.943747
350482,26181.790610,2613,2481,134.099307,955.698874,-0.243714,3.314662,13,154.536712,900.782107,154.668771,900.760422,220,20.437405,58.596406
350555,26181.790610,2613,2589,134.099307,955.698874,-0.243714,3.314662,13,123.410447,1084.791858,122.880767,1090.299428,220,-10.688861,129.534745


In [109]:
# Copie de la bdd pour récuperer les colonnes sans impacter la vraie base de données

data_copy = df.copy()
data_copy = data_copy.drop(["label"], axis=1)

columns = data_copy.columns
columns

Index(['sendTime', 'sender', 'receiver', 'pos_x_send', 'pos_y_send',
       'spd_x_send', 'spd_y_send', 'pos_x_rec', 'pos_y_rec', 'pos_x_rec_f',
       'pos_y_rec_f', 'nb_packets_sent', 'distRealSR1', 'distance'],
      dtype='object')

Amélioration à faire :

Séléctionner seulement les sender avec value_counts > 20

Séparation de la base de données : diviser pour régner

In [9]:
# /!\ attention à bien utiliser la base de données rangée par sender pour limiter la perte de données
# Indice égal à 0, 1 ou 2
# Célia = 0
# Chléo = 1
# Thomas = 2

def split_data_in_3(df, indice):
    # Division de la bdd en 3
    split_data = np.array_split(df,3)

    # Division en plusieurs sous-bdd pour diminuer le temps de calcul par exécution
    #divided_data = np.array_split(split_data[indice],50)

    return split_data

/!\ à bien changer l'indice !!

In [10]:
divided_dataframes = split_data_in_3(df.sort_values("sender"),0)

In [11]:
len(divided_dataframes)

3

In [12]:
sequence = (df.loc[df['sender'] == senders[0]].sort_values("sendTime"))[0:20]
sequence.values.tolist()


[[25212.6028766769,
  9.0,
  45.0,
  257.604241968877,
  141.917135558921,
  -2.40990815416535,
  14.3161890692871,
  0.0,
  265.540271074589,
  46.2074344300037,
  268.53985521445,
  49.8010865575598,
  96.0,
  7.93602910571184,
  96.0381562096726],
 [25213.6028774223,
  9.0,
  45.0,
  255.242942868568,
  156.206787123968,
  -2.56664045650258,
  14.2100266004694,
  0.0,
  268.53985521445,
  49.8010865575598,
  268.05097732445,
  55.3313015392878,
  96.0,
  13.2969123458828,
  107.233301688246],
 [25214.6028782208,
  9.0,
  45.0,
  252.85775070182,
  170.495740224512,
  -2.34282858790929,
  14.2543322960901,
  0.0,
  268.05097732445,
  55.3313015392878,
  270.113953171394,
  63.3182934031594,
  96.0,
  15.1932266226306,
  116.162309175091],
 [25215.6028790598,
  9.0,
  45.0,
  250.52554382896,
  184.83295234553,
  -2.35287357908116,
  14.3150019903528,
  0.0,
  270.113953171394,
  63.3182934031594,
  268.836895587202,
  72.9150644119116,
  96.0,
  19.5884093424332,
  123.083378725348],

In [10]:
df.loc[df['sender'] == 2613]['label'].tolist()

[13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,
 13,


Création des séquences

In [126]:
def sequence_creation(df):
    senders_sequences = np.array()
    senders_label = np.array
    for sender in senders:
        sender_data_sorted = df.loc[df['sender'] == sender].sort_values("sendTime")
        sender_data_sorted = sender_data_sorted.drop(["label","sender"], axis=1)
        sequence_array = np.array

        #print(sender_data_sorted.shape)

        length = df.loc[df['sender'] == sender].shape[0]
        slide = 10
        start = 0
        end = 20

        # On récupère la valeur du label pour ce sender
        label = df.loc[df['sender'] == sender]['label'].tolist()[0]
        
        if (length >= 20):
            senders_label = np.append(senders_label,label) # ajout du label pour les prochaines séquences

        # On vérifie qu'il est possible de faire une séquence de taille 20
        while length >= 20:
            # Extraction par tranche de 20 avec une inter de 10
            sequence = sender_data_sorted[start:end]

            # Labels correspondant
            #labels =  pd.Series.tolist(sequence["label"])

            # On transforme les 13 en 1, cette formule marche toujours si on met d'autres types d'attaques
            #labels[:] = [x if x == 0 else 1 for x in labels]

            # Attribution des tableaux numpy
            sequence_array = np.append(sequence_array, np.array(sequence.values, dtype=object))
            #label_array.append(labels)

            # Mise à jour des variables
            start += slide
            end += slide
            length -= 10
        
        if len(sequence_array)>0:
            senders_sequences = np.append(senders_sequences, np.array(sequence_array, dtype=object))

    
    return senders_sequences, senders_label

/!\ à bien changer l'indice !!

In [127]:
sequence_test, label_test = sequence_creation(df)

TypeError: array() missing required argument 'object' (pos 0)

In [112]:
len(sequence_test)

1112

Vérification des séquences obtenues

Test RNN

In [113]:
import pandas as pd
import numpy as np
from tensorflow import keras
from keras import layers, applications, utils, models, optimizers, Input, callbacks
from keras.models import Sequential
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns

In [114]:
# Transformation en array numpy
X = np.array(sequence_test, dtype=object)
y = np.array(label_test)

In [115]:
# Séparation en données d'entrainement et de test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)

In [116]:
print("X_train : ", X_train.shape)
print("y_train : ", y_train.shape)
print("X_test : ", X_test.shape)
print("y_test : ", y_test.shape)

X_train :  (1000,)
y_train :  (1000,)
X_test :  (112,)
y_test :  (112,)


In [117]:
X_train = X_train[:20]
y_train = y_train[:20]

In [125]:
X_train.shape

(20,)

In [120]:
simple_model = keras.Sequential([
    layers.Dense(32, input_shape=(X_train.shape[0], None, 20,12), activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(512, activation='relu'),
    layers.BatchNormalization(),
    layers.Dense(128, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

In [121]:
callback = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)

In [122]:
opt = keras.optimizers.SGD(learning_rate=0.1)
simple_model.compile(loss='mean_squared_error', optimizer='adam', metrics='accuracy')
simple_model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_18 (Dense)            (None, 20, None, 20, 32)  416       
                                                                 
 batch_normalization_8 (Batc  (None, 20, None, 20, 32)  128      
 hNormalization)                                                 
                                                                 
 dense_19 (Dense)            (None, 20, None, 20, 512  16896     
                             )                                   
                                                                 
 batch_normalization_9 (Batc  (None, 20, None, 20, 512  2048     
 hNormalization)             )                                   
                                                                 
 dense_20 (Dense)            (None, 20, None, 20, 128  65664     
                             )                        

In [123]:
history = simple_model.fit(X_train, y_train, epochs=10, batch_size=128, verbose=2, validation_split=0.3)

ValueError: Failed to convert a NumPy array to a Tensor (Unsupported object type numpy.ndarray).

In [124]:
print("INPUT")
[print(i.shape, i.dtype) for i in simple_model.inputs]
print("OUTPUT")
[print(o.shape, o.dtype) for o in simple_model.outputs]
print("LAYERS")
[print(l.name, l.input_shape, l.dtype) for l in simple_model.layers]

INPUT
(None, 20, None, 20, 12) <dtype: 'float32'>
OUTPUT
(None, 20, None, 20, 1) <dtype: 'float32'>
LAYERS
dense_18 (None, 20, None, 20, 12) float32
batch_normalization_8 (None, 20, None, 20, 32) float32
dense_19 (None, 20, None, 20, 32) float32
batch_normalization_9 (None, 20, None, 20, 512) float32
dense_20 (None, 20, None, 20, 512) float32
dense_21 (None, 20, None, 20, 128) float32


[None, None, None, None, None, None]